In [4]:
import pandas as pd

In [5]:
zip_file = '../data/zipcodes.csv'
crime_file = '../data/south_carolina_crime_report.csv'
school_rating_file = '../data/School_rating_zipcode.csv'
job_growth_file = '../data/job_growth_data_w_lat_long.csv'
property_file = '../data/final_property_data.csv'

In [9]:
def merge_file (zip_df, property_df, school_df, crime_df, job_growth_df):
    output_df = pd.merge(zip_df[["zip","latitude","longitude"]],property_df, on="zip",  how="right" )
    output_df = pd.merge(output_df,school_df,  on="zip" )
    output_df = pd.merge(output_df,crime_df,  on="zip" )
    output_df = output_df.fillna(value=0)
    output_df = pd.merge(output_df,job_growth_df[["zip","job_growth_per"]],  on="zip" )
    return output_df

zip_df = pd.read_csv(zip_file,delimiter=',')
property_df = pd.read_csv(property_file,delimiter=',')
school_df = pd.read_csv(school_rating_file,delimiter=',')
crime_df = pd.read_csv(crime_file,delimiter=',')
job_growth_df = pd.read_csv(job_growth_file,delimiter=',')



property_df.drop(['Unnamed: 0','Ratio'],axis=1,inplace=True)
property_df.rename(columns={"Zip":"zip","Rental Value 2016":"Rent"},inplace=True)
property_df["Rent"] = property_df.Rent.astype(int)

school_df.drop(['Unnamed: 0'],axis=1,inplace=True)
school_df.rename(columns={"rating":"School Rating"},inplace=True)

crime_df = pd.DataFrame(crime_df.groupby('Zip')['Total'].sum())
crime_df.reset_index(inplace=True)
crime_df.rename(columns={"Zip":"zip","Total":"Total Crimes"},inplace=True)

job_growth_df.drop(['Unnamed: 0'],axis=1,inplace=True)
job_growth_df.rename(columns={"?zip":"zip"},inplace=True)

final_df = merge_file(zip_df, property_df, school_df, crime_df, job_growth_df)

final_df = final_df[final_df["Rent"] != 0]
final_df["Normalized Ratio"] = round(final_df["Normalized Ratio"],2)
final_df.to_csv("../data/final_investment_criteria.csv",index=False)
final_df.head()

,zip,latitude,longitude,Cost of Single Family Home,Rent,Normalized Ratio,School Rating,Total Crimes,job_growth_per
0,29003,33.29,-81.03,208300,719,0.84,4.7,185,-0.7
1,29009,34.41,-80.34,80700,705,2.11,4.0,5,2.1
2,29010,34.21,-80.24,208300,694,0.81,2.3,311,2.1
3,29018,33.34,-80.68,81000,612,1.83,0.0,35,2.1
4,29020,34.26,-80.61,110400,719,1.58,4.2,605,2.1
